# Simulation of an ideal gas

In [ ]:
# importing some modules
%matplotlib inline 
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
# we import the Container object from matplotlib as it will later be useful to collect the Artist objects from the scatter plot and the histogram
# this Container can then be easily passed to the ArtistAnimation object to obtain a frame by frame animation
from matplotlib.container import Container
import matplotlib.animation as animation
# tqdm is a module which incorperates a useful progress bar
from tqdm import tqdm

### Plotting the initial configuration

In [ ]:
# specify some initial parameters such as "number of particles", "initial configuration", "initial velocities" and length of the quadratic box
n_particles: int = 500
trj_init = np.random.uniform(low=35, high=40, size=(n_particles, 2))
vel_init = np.random.uniform(low=-20, high=-20, size=(n_particles, 2))
length = 100

In [ ]:
# to get a quick overview of the point particle's initial configuration together with its velocity vectors, we plot
fig, ax = plt.subplots(figsize=(5,5))
for (r, v) in zip(trj_init, vel_init):
    ax.quiver(r[0], r[1], v[0], v[1], color='b', scale=None, width=0.003);
    ax.plot(r[0], r[1], 'o', color='gray', markersize=3);
    ax.set_xlim(0, length)
    ax.set_ylim(0, length)
    plt.tight_layout()
plt.show()
plt.close()

## Adding the time evolution and the boundary conditions

In [ ]:
# as we update the trajectory of every point particle in time steps dt, it is necessary to specify the updates in a step function
def step_func(trj, vel, dt, mode):
    # periodic boundaries makes use of the mod length
    if mode == "periodic":
        # periodic boundary using mod length
        trj = (trj + vel*dt)%length
    # reflective boundary requires to flip the velocity's components respectively
    elif mode == "reflective":
        for i in range(len(trj[:, 0])):
            if trj[i, 0] <= 0:
                vel[i, 0] = -vel[i, 0]
            elif trj[i, 0] >= length:
                vel[i, 0] = -vel[i, 0]
            elif trj[i, 1] <= 0:
                vel[i, 1] = -vel[i, 1]
            elif trj[i, 1] >= length:
                vel[i, 1] = -vel[i, 1]
        trj = trj + vel*dt
        return trj, vel

In [ ]:
# as we constantly update the trajectory after the plotting and saving, it is crucial to set trj to the initial trajecctory
trj = trj_init

frames = []
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
plt.tight_layout()
ax.set_xlim(0, length)
ax.set_ylim(0, length)

# for every simulation step, plot the point particles
for step in range(100):
    scatter = ax.scatter(x=trj[:, 0], y=trj[:, 1], s=5, c="gray", clip_on=True)
    
    # whatever we append to frames must be iterable
    # otherwise the ArtistAnimation's ._init_draw() function can't iterate over all artists in one frame and raises a TypeError
    frames.append([scatter])
    plt.close()
    trj, vel = step_func(trj, vel_init, 0.05, mode="reflective")


In [ ]:
ani = animation.ArtistAnimation(fig, frames, interval=24, blit=False)
ani.save("ani.mp4")

## Equipping the point particles with a volume and implementing elastic collisions

In [ ]:
# firstly, we want to update the trajectories of all particles, whose absolute trajectory difference is smaller than their collective radii
# secondly, after we gave every particle a volume, we can now calculate the velocity of the ith particle after the collision with the jth particle
def get_collisions(n_particles, trj, vel):
    for i in range(n_particles):
        for j in range(n_particles):
            # compute the absolute distance between the ith and the jth particle
            dist = np.linalg.norm(trj[i, :] - trj[j, :])
            # compute the collective radius of compound particle consisting of the ith and the jth particle
            radii = rad[i] + rad[j]
            
            if dist <= radii:
                # here we register a collision event between the ith and the jth particle
                # calculate the normal between the two particles
                # using nan_to_num here is fine because dividing the zero vector by zero should give zero in our physical case
                normal = np.nan_to_num((trj[i, :] - trj[j, :])/dist)
                # calculate the overlap distance between the two particles
                overlap = radii - dist
                # shift each particle half of the overlap distance in opposite directions
                trj[i, :] += normal*overlap/2
                trj[j, :] -= normal*overlap/2
                # compute the new absolute distances
                # use nan_to_num to turn all nan to 0 (if a collision is too violent, it would kick the particle out of the frame)
                dist = np.linalg.norm(trj[i, :] - trj[j, :])
                vel[i, :] -= np.nan_to_num(np.dot((vel[i, :] - vel[j, :]), (trj[i, :] - trj[j, :]))/dist**2) *  (trj[i, :] - trj[j, :])
                #vel[j, :] += np.nan_to_num(np.dot((vel[j, :] - vel[i, :]), (trj[j, :] - trj[i, :]))/dist**2) *  (trj[j, :] - trj[i, :])
    return trj, vel

In [ ]:
# as we update the trajectory of every point particle in time steps dt, it is necessary to specify the updates in a step function
def step_func(trj, vel, dt, mode):
    trj, vel = get_collisions(n_particles, trj, vel)
    # periodic boundaries makes use of the mod length
    if mode == "periodic":
        # periodic boundary using mod length
        trj = (trj + vel*dt)%length
    # reflective boundary requires to flip the velocity's components respectively
    elif mode == "reflective":
        for i in range(len(trj[:, 0])):
            if trj[i, 0] <= rad[i]:
                vel[i, 0] = -vel[i, 0]
            elif trj[i, 0] >= length - rad[i]:
                vel[i, 0] = -vel[i, 0]
            elif trj[i, 1] <= rad[i]:
                vel[i, 1] = -vel[i, 1]
            elif trj[i, 1] >= length - rad[i]:
                vel[i, 1] = -vel[i, 1]
        trj = trj + vel*dt
    return trj, vel

In [ ]:
# as we constantly update the trajectory after the plotting and saving, it is crucial to set trj to the initial trajecctory
trj = trj_init
vel = vel_init

# we give each particle a radius
rad = np.random.uniform(low=1, high=1, size=n_particles)

frames = []
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
plt.tight_layout()
ax.set_xlim(0, length)
ax.set_ylim(0, length)

# for every simulation step, plot the point particles
for step in range(1000):
    scatter = ax.scatter(x=trj[:, 0], y=trj[:, 1], s=4*rad**2, c=rad, clip_on=True)
    
    # whatever we append to frames must be iterable
    # otherwise the ArtistAnimation's ._init_draw() function can't iterate over all artists in one frame and raises a TypeError
    frames.append([scatter])
    plt.close()
    trj, vel = step_func(trj, vel, 0.01, mode="periodic")

In [ ]:
ani = animation.ArtistAnimation(fig, frames, interval=30, blit=False)
ani.save("ani.mp4")

## Adding an impulse histogram

In [ ]:
trj = trj_init
vel = vel_init

# we give each particle a radius
rad = np.random.uniform(low=1, high=1, size=n_particles)

frames = []
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
#plt.tight_layout()
ax[0].set_xlim(0, length)
ax[0].set_ylim(0, length)
ax[1].set_xlabel("Velocities [m/s]")
ax[1].set_ylabel("Absolute frequency")
for step in range(1000):
    scatter = ax[0].scatter(x=trj[:, 0], y=trj[:, 1], s=rad**2, c="gray", clip_on=True)    
    n, bins, patches = ax[1].hist(vel[:, 1], bins=100, histtype="bar", color="grey")
    objects = patches.get_children()
    objects.append(scatter)
    
    container = Container(objects)
    
    frames.append(container)
    plt.close()
    
    trj, vel = step_func(trj, vel, 0.05, mode="reflective")

In [ ]:
ani = animation.ArtistAnimation(fig, frames, interval=30, blit=False)
ani.save("ani.mp4")

## Counting the wall collisions to determine the system's pressure

## Confining the particles over time